# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [3]:
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations

from tqdm import tqdm_notebook

%autosave 5
%matplotlib inline

Using TensorFlow backend.


Autosaving every 5 seconds


## Fetching Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCat](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [4]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [5]:
from datetime import datetime
import pandas as pd
historic_data = pd.read_csv('bitcoin_cash_price.csv')
historic_data.columns=historic_data.columns.str.lower()
historic_data['date'] = historic_data['date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d'))

The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [21]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` variable.
#
model_data=historic_data[(historic_data['date'] > '2017-01-01') & (historic_data['date'] < '2018-01-07')]
model_data.tail()

,date,open,high,low,close,volume,market cap
208,2017-07-27,417.10,460.97,367.78,385.48,"533,207",-
209,2017-07-26,407.08,486.16,321.79,365.82,"1,784,640",-
210,2017-07-25,441.35,541.66,338.09,406.90,"524,908",-
211,2017-07-24,412.58,578.89,409.21,440.70,"190,952",-
212,2017-07-23,555.89,578.97,411.78,413.06,"85,013",-


## The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [22]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [23]:
M.build()

In [24]:
M.train(epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 0.0109
Epoch 2/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0093
Epoch 3/100
1/1 [==============================] - 0s 24ms/step - loss: 0.0083
Epoch 4/100
1/1 [==============================] - 0s 19ms/step - loss: 0.0076
Epoch 5/100
1/1 [==============================] - 0s 20ms/step - loss: 0.0070
Epoch 6/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0064
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 0.0060
Epoch 8/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0056
Epoch 9/100
1/1 [==============================] - 0s 24ms/step - loss: 0.0052
Epoch 10/100
1/1 [==============================] - 0s 23ms/step - loss: 0.0048
Epoch 11/100
1/1 [==============================] - 0s 25ms/step - loss: 0.0045
Epoch 12/100
1/1 [==============================] - 0s 21ms/step - loss: 0.0042
Epoch 13/100
1/1 [==============================] -

We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [25]:
M.predict(denormalized=True)

array([2789.5293, 2946.4119, 2784.9512, 2644.3496, 2941.4678, 3185.1318,
       2987.2695], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [26]:
M.evaluate()

{'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}

Finally, we can now save the trained model on disk for later use.

In [27]:
M.save('bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [28]:
M = Model(path='bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [29]:
M.predict(denormalized=True)

array([2789.5293, 2946.4119, 2784.9512, 2644.3496, 2941.4678, 3185.1318,
       2987.2695], dtype=float32)

## New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [44]:
M1 = Model(data=model_data[0*7:7*21 + 7],
          variable='close',
          predicted_period_size=7)

In [45]:
M1.build()

In [46]:
M1.train()

In [48]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
results = []
for i in range(0, 11):
    M1.train(model_data[0*7+i:i+7*21+7])
    results.append(M.evaluate())

In [49]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

Week 41: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 42: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 43: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 44: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 45: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 46: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 47: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 48: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 49: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 50: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}
Week 51: {'mse': 0.0, 'rmse': 196.34, 'mape': 5.42}


In [50]:
M1.predict(denormalized=True)

array([2924.0322, 2402.0068, 1869.2751, 2194.3906, 2488.8208, 2634.784 ,
       2495.2947], dtype=float32)

## New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [51]:
old_data = model_data[0*7:7*48 + 7]

In [52]:
new_data = model_data[0*7:7*49 + 7]

In [53]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [54]:
M2.build()
M2.train()

In [55]:
M2.predict(denormalized=True)

array([2778.094 , 2948.135 , 2786.0022, 2656.8098, 2947.5115, 3187.037 ,
       2988.4014], dtype=float32)

Now, assume that new data is available. Using this technique we go ahead and create a new model using only the new data. 

In [56]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
M3=Model(data=new_data,variable='close',predicted_period_size=7)


In [57]:
M3.build()
M3.train()

In [58]:
M3.predict(denormalized=True)

array([2782.0972, 2941.466 , 2782.4417, 2656.0046, 2955.2495, 3202.3953,
       2998.596 ], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.